### Import libraries

In [1]:
from IPython.core.display import display
from src.utils.preprocessing import preprocessing_pipeline
from src.utils.get_data import import_data, split_experts
from src.utils.train import hyperparameter_tuning_cv
from src.utils.config import *
import pandas as pd
import numpy as np

In [2]:
%load_ext autoreload
%autoreload 2

### Load Data

In [3]:
DATA_PATH = '../../../data'
X_fine, y_fine = import_data(DATA_PATH, segmentation_type='fine',
                             drop_user_features=False,
                             drop_expert=True)

In [4]:
# For each expert separately
X_e, y_e = import_data(DATA_PATH, segmentation_type='fine',
                       drop_user_features=False,
                       drop_expert=False)

In [5]:
display(X_fine.head())
display(y_fine.head())

EEPD50_100  EEPD100_150  \
subject                              file_id                            
008ba489-31ad-44d8-856b-fcf72369dc46 0               6.0          6.0   
                                     1               3.0          3.0   
                                     2               4.0          4.0   
                                     3               4.0          4.0   
                                     4               4.0          4.0   

                                              EEPD150_200  EEPD200_250  \
subject                              file_id                             
008ba489-31ad-44d8-856b-fcf72369dc46 0                5.0          4.0   
                                     1                3.0          4.0   
                                     2                3.0          4.0   
                                     3                4.0          4.0   
                                     4                5.0          4.0   

                                              EEPD250_300  EEPD300_350  \
subject                              file_id                             
008ba489-31ad-44d8-856b-fcf72369dc46 0                5.0          5.0   
                                     1                4.0          5.0   
                                     2                3.0          3.0   
                                     3                4.0          5.0   
                                     4                4.0          5.0   

                                              EEPD350_400  EEPD400_450  \
subject                              file_id                             
008ba489-31ad-44d8-856b-fcf72369dc46 0                6.0          6.0   
                                     1                4.0          4.0   
                                     2                4.0          4.0   
                                     3                3.0          4.0   
                                     4                3.0          4.0   

                                              EEPD450_500  EEPD500_550  ...  \
subject                              file_id                            ...   
008ba489-31ad-44d8-856b-fcf72369dc46 0                5.0          6.0  ...   
                                     1                4.0          3.0  ...   
                                     2                4.0          4.0  ...   
                                     3                4.0          3.0  ...   
                                     4                4.0          3.0  ...   

                                              PSD_225-425  PSD_450-550  \
subject                              file_id                             
008ba489-31ad-44d8-856b-fcf72369dc46 0           0.398299     0.026407   
                                     1           0.230052     0.086242   
                                     2           0.105747     0.180344   
                                     3           0.235517     0.121967   
                                     4           0.230394     0.119689   

                                              PSD_1325-1600  PSD_1600-1900  \
subject                              file_id                                 
008ba489-31ad-44d8-856b-fcf72369dc46 0             0.013356       0.033366   
                                     1             0.023806       0.008824   
                                     2             0.034701       0.006258   
                                     3             0.061882       0.002742   
                                     4             0.069418       0.002717   

                                              PSD_2500-2900  PSD_3100-3700  \
subject                              file_id                                 
008ba489-31ad-44d8-856b-fcf72369dc46 0             0.043195       0.010385   
                                     1             0.056102       0.011429   
                                     2       

Label
subject                              file_id       
008ba489-31ad-44d8-856b-fcf72369dc46 0          1.0
                                     1          1.0
                                     2          1.0
                                     3          1.0
                                     4          1.0

### Preprocessing

In [6]:
X_fine = preprocessing_pipeline(X_fine)

In [7]:
# Save the expert feature for split
expert = X_e['Expert'].copy()
X_e = preprocessing_pipeline(X_e)
X_e['Expert'] = expert.values

### Split expert models

In [8]:
X_e_1, y_e_1, X_e_2, y_e_2, X_e_3, y_e_3 = split_experts(X_e, y_e)

### Grid search

In [9]:
DECISION_METRIC = 'roc_auc_score'
CV_K = 10

#### 1. Logistic regression

In [10]:
log_results = hyperparameter_tuning_cv(model='logistic', data=X_fine, labels=y_fine.Label, cv_k=CV_K,
                                       params=LOGISTIC_PARAMS)

display(log_results)

f1_score  roc_auc_score  roc_auc_score_var  \
max_iter oversampling                                               
10000    True          0.404850       0.577510           0.000112   
         False         0.145581       0.527084           0.000046   
100000   True          0.404850       0.577510           0.000112   
         False         0.145581       0.527084           0.000046   

                       accuracy_score  explained_variance_score  
max_iter oversampling                                            
10000    True                0.623538                 -0.871594  
         False               0.732002                 -0.114466  
100000   True                0.623538                 -0.871594  
         False               0.732002                 -0.114466

In [11]:
# Best model parameters

best_log = log_results.iloc[[
    log_results.reset_index()[DECISION_METRIC].idxmax()]]

display(best_log)

,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
max_iter,oversampling,,,,,
10000,True,0.40485,0.57751,0.000112,0.623538,-0.871594


In [12]:
# for each expert
log_results_1 = hyperparameter_tuning_cv(model='logistic', data=X_e_1, labels=y_e_1, cv_k=CV_K,
                                         params=LOGISTIC_PARAMS)
log_results_2 = hyperparameter_tuning_cv(model='logistic', data=X_e_2, labels=y_e_2, cv_k=CV_K,
                                         params=LOGISTIC_PARAMS)
log_results_3 = hyperparameter_tuning_cv(model='logistic', data=X_e_3, labels=y_e_3, cv_k=CV_K,
                                         params=LOGISTIC_PARAMS)

In [13]:
# Best model parameters

best_log_1 = log_results_1.iloc[[
    log_results_1.reset_index()[DECISION_METRIC].idxmax()]]
best_log_2 = log_results_2.iloc[[
    log_results_2.reset_index()[DECISION_METRIC].idxmax()]]
best_log_3 = log_results_3.iloc[[
    log_results_3.reset_index()[DECISION_METRIC].idxmax()]]

display(best_log_1, best_log_2, best_log_3)

,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
max_iter,oversampling,,,,,
10000,True,0.377124,0.651013,0.000329,0.734684,-0.951475


,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
max_iter,oversampling,,,,,
10000,True,0.285395,0.544892,0.000432,0.642126,-1.264211


,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
max_iter,oversampling,,,,,
10000,False,0.624511,0.654648,0.000215,0.657241,-0.371622


#### 2. Linear Discriminant Analysis

In [14]:
lda_results = hyperparameter_tuning_cv(model='lda', data=X_fine, labels=y_fine.Label, cv_k=CV_K,
                                       params=LDA_PARAMS)

display(lda_results)

,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
oversampling,,,,,
True,0.404396,0.577981,0.000103,0.626153,-0.862761
False,0.155366,0.528930,0.000036,0.731479,-0.126611


In [15]:
# Best model parameters

best_lda = lda_results.iloc[[
    lda_results.reset_index()[DECISION_METRIC].idxmax()]]

display(best_lda)

,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
oversampling,,,,,
True,0.404396,0.577981,0.000103,0.626153,-0.862761


In [16]:
# For each expert
lda_results_1 = hyperparameter_tuning_cv(model='lda', data=X_e_1, labels=y_e_1, cv_k=CV_K,
                                         params=LDA_PARAMS)
lda_results_2 = hyperparameter_tuning_cv(model='lda', data=X_e_2, labels=y_e_2, cv_k=CV_K,
                                         params=LDA_PARAMS)
lda_results_3 = hyperparameter_tuning_cv(model='lda', data=X_e_3, labels=y_e_3, cv_k=CV_K,
                                         params=LDA_PARAMS)

In [17]:
best_lda_1 = lda_results_1.iloc[[
    lda_results_1.reset_index()[DECISION_METRIC].idxmax()]]
best_lda_2 = lda_results_2.iloc[[
    lda_results_2.reset_index()[DECISION_METRIC].idxmax()]]
best_lda_3 = lda_results_3.iloc[[
    lda_results_3.reset_index()[DECISION_METRIC].idxmax()]]

display(best_lda_1, best_lda_2, best_lda_3)

,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
oversampling,,,,,
True,0.371434,0.647913,0.000616,0.724959,-1.004229


,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
oversampling,,,,,
True,0.282779,0.542874,0.000426,0.641995,-1.268573


,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
oversampling,,,,,
False,0.626488,0.658413,0.000185,0.661379,-0.352997


#### 3. K-nearest Neighbors

In [18]:
knn_results = hyperparameter_tuning_cv(model='knn', data=X_fine, labels=y_fine.Label, cv_k=CV_K,
                                       params=KNN_PARAMS)

display(knn_results)

f1_score  roc_auc_score  roc_auc_score_var  \
n_neighbors oversampling                                               
1           True          0.532269       0.679682           0.000148   
            False         0.514598       0.668910           0.000083   
2           True          0.520542       0.670757           0.000092   
            False         0.319033       0.583493           0.000021   
3           True          0.503214       0.647867           0.000107   
            False         0.425737       0.619078           0.000060   
4           True          0.504154       0.652069           0.000113   
            False         0.289100       0.572579           0.000033   
5           True          0.491147       0.631694           0.000070   
            False         0.383670       0.603249           0.000074   
6           True          0.492263       0.636623           0.000120   
            False         0.249418       0.559468           0.000048   
7           True          0.481324       0.617750           0.000110   
            False         0.328611       0.582763           0.000041   
8           True          0.483615       0.624173           0.000119   
            False         0.216402       0.549327           0.000041   
9           True          0.473583       0.606222           0.000114   
            False         0.278500       0.565033           0.000066   
10          True          0.475333       0.611976           0.000077   
            False         0.194863       0.543868           0.000048   
11          True          0.468232       0.597725           0.000089   
            False         0.246486       0.556335           0.000036   
12          True          0.472008       0.605866           0.000064   
            False         0.169034       0.537606           0.000059   
13          True          0.463129       0.589638           0.000030   
            False         0.211778       0.546815           0.000059   
14          True          0.467139       0.598163           0.000036   
            False         0.145337       0.530546           0.000055   
15          True          0.461582       0.586250           0.000057   
            False         0.191240       0.541440           0.000072   

                          accuracy_score  explained_variance_score  
n_neighbors oversampling                                            
1           True                0.660105                 -0.547510  
            False               0.747551                 -0.286397  
2           True                0.697575                 -0.496121  
            False               0.754779                 -0.092474  
3           True                0.579933                 -0.652730  
            False               0.738041                 -0.303014  
4           True                0.619353                 -0.666792  
            False               0.750262                 -0.099574  
5           True                0.540799                 -0.655810  
            False               0.746315                 -0.214427  
6           True                0.574560                 -0.703280  
            False               0.745934                 -0.096528  
7           True                0.512696                 -0.653145  
            False               0.745126                 -0.174490  
8           True                0.542844                 -0.707422  
            False               0.742653                 -0.092671  
9           True                0.490300                 -0.644469  
            False               0.740276                 -0.166258  
10          True                0.515977                 -0.704194  
            False               0.742130                 -0.077689  
11          True                0.473371                 -0.630259  
            False               0.740989                 -0.132020  
12          True                0.498764                 -0.68240

In [19]:
# Best model parameters

best_knn = knn_results.iloc[[
    knn_results.reset_index()[DECISION_METRIC].idxmax()]]

display(best_knn)

,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
n_neighbors,oversampling,,,,,
1,True,0.532269,0.679682,0.000148,0.660105,-0.54751


In [20]:
knn_results_1 = hyperparameter_tuning_cv(
    model='knn', data=X_e_1, labels=y_e_1, cv_k=CV_K, params=KNN_PARAMS)
knn_results_2 = hyperparameter_tuning_cv(
    model='knn', data=X_e_2, labels=y_e_2, cv_k=CV_K, params=KNN_PARAMS)
knn_results_3 = hyperparameter_tuning_cv(
    model='knn', data=X_e_3, labels=y_e_3, cv_k=CV_K, params=KNN_PARAMS)

In [21]:
display(knn_results_1, knn_results_2, knn_results_3)

f1_score  roc_auc_score  roc_auc_score_var  \
n_neighbors oversampling                                               
1           True          0.462009       0.745942           0.000359   
            False         0.551431       0.738122           0.000399   
2           True          0.480965       0.744827           0.000619   
            False         0.362173       0.612809           0.000221   
3           True          0.406918       0.715558           0.000392   
            False         0.429183       0.648188           0.000214   
4           True          0.425558       0.726707           0.000408   
            False         0.257138       0.573406           0.000213   
5           True          0.383374       0.698203           0.000327   
            False         0.332101       0.601547           0.000221   
6           True          0.395040       0.705580           0.000294   
            False         0.178153       0.547790           0.000268   
7           True          0.364119       0.677469           0.000341   
            False         0.241817       0.567716           0.000181   
8           True          0.378784       0.692830           0.000279   
            False         0.133459       0.534265           0.000165   
9           True          0.354832       0.667759           0.000303   
            False         0.178066       0.547534           0.000223   
10          True          0.368098       0.683241           0.000461   
            False         0.095846       0.523824           0.000116   
11          True          0.346930       0.658428           0.000228   
            False         0.139583       0.536187           0.000208   
12          True          0.356100       0.669158           0.000285   
            False         0.063002       0.515497           0.000198   
13          True          0.337168       0.644365           0.000245   
            False         0.094363       0.523380           0.000214   
14          True          0.345449       0.655001           0.000276   
            False         0.050709       0.512297           0.000103   
15          True          0.330517       0.634361           0.000144   
            False         0.063481       0.515256           0.000170   

                          accuracy_score  explained_variance_score  
n_neighbors oversampling                                            
1           True                0.727391                 -0.800967  
            False               0.861912                 -0.072726  
2           True                0.766451                 -0.649248  
            False               0.871475                  0.084406  
3           True                0.629011                 -1.084553  
            False               0.863695                 -0.026923  
4           True                0.669368                 -1.000436  
            False               0.862885                  0.042322  
5           True                0.569368                 -1.142860  
            False               0.863371                  0.011558  
6           True                0.607293                 -1.130533  
            False               0.857536                  0.023534  
7           True                0.524959                 -1.171022  
            False               0.858671                  0.006270  
8           True                0.561588                 -1.156439  
            False               0.854295                  0.008748  
9           True                0.494165                 -1.143212  
            False               0.855754                  0.005583  
10          True                0.527391                 -1.146814  
            False               0.852350                  0.003596  
11          True                0.468720                 -1.113262  
            False               0.854295                  0.005964  
12          True                0.499514                 -1.15209

f1_score  roc_auc_score  roc_auc_score_var  \
n_neighbors oversampling                                               
1           True          0.453041       0.691204           0.000259   
            False         0.457131       0.661797           0.000189   
2           True          0.458492       0.684557           0.000318   
            False         0.206071       0.553508           0.000128   
3           True          0.417643       0.669078           0.000275   
            False         0.317358       0.591331           0.000267   
4           True          0.423674       0.669883           0.000173   
            False         0.158014       0.539746           0.000154   
5           True          0.396872       0.649071           0.000207   
            False         0.199622       0.549632           0.000227   
6           True          0.400512       0.650397           0.000302   
            False         0.070540       0.515354           0.000045   
7           True          0.385574       0.636022           0.000202   
            False         0.101533       0.521543           0.000083   
8           True          0.385470       0.634400           0.000293   
            False         0.023931       0.504408           0.000027   
9           True          0.371993       0.617967           0.000188   
            False         0.050829       0.510476           0.000052   
10          True          0.378961       0.626839           0.000270   
            False         0.011021       0.501918           0.000008   
11          True          0.367924       0.612241           0.000288   
            False         0.025932       0.505109           0.000026   
12          True          0.371835       0.617627           0.000235   
            False         0.006976       0.501371           0.000005   
13          True          0.362116       0.603811           0.000168   
            False         0.013711       0.502934           0.000012   
14          True          0.365834       0.609310           0.000253   
            False         0.006880       0.501586           0.000008   
15          True          0.356192       0.594637           0.000173   
            False         0.009628       0.502047           0.000010   

                          accuracy_score  explained_variance_score  
n_neighbors oversampling                                            
1           True                0.697507                 -0.787936  
            False               0.822835                 -0.166084  
2           True                0.743438                 -0.633940  
            False               0.824541                 -0.023151  
3           True                0.600000                 -0.988632  
            False               0.820604                 -0.121011  
4           True                0.643570                 -0.957290  
            False               0.823360                 -0.008595  
5           True                0.544094                 -1.034880  
            False               0.818373                 -0.076306  
6           True                0.580184                 -1.062740  
            False               0.816798                 -0.023492  
7           True                0.507218                 -1.028833  
            False               0.814829                 -0.055658  
8           True                0.536877                 -1.096216  
            False               0.814961                 -0.017517  
9           True                0.475066                 -1.040256  
            False               0.815748                 -0.022936  
10          True                0.508793                 -1.082871  
            False               0.815223                 -0.008779  
11          True                0.455906                 -1.009016  
            False               0.815486                 -0.013539  
12          True                0.481627                 -1.06276

f1_score  roc_auc_score  roc_auc_score_var  \
n_neighbors oversampling                                               
1           True          0.721224       0.734365           0.000204   
            False         0.714848       0.735849           0.000147   
2           True          0.612875       0.685581           0.000123   
            False         0.563625       0.668846           0.000183   
3           True          0.696809       0.704123           0.000275   
            False         0.677792       0.705249           0.000175   
4           True          0.641335       0.687951           0.000217   
            False         0.594369       0.676623           0.000197   
5           True          0.681227       0.685481           0.000340   
            False         0.662385       0.693842           0.000207   
6           True          0.641834       0.677860           0.000176   
            False         0.599456       0.673818           0.000159   
7           True          0.672150       0.676085           0.000310   
            False         0.647161       0.682820           0.000193   
8           True          0.644171       0.674575           0.000185   
            False         0.588975       0.664593           0.000140   
9           True          0.667618       0.669894           0.000165   
            False         0.633689       0.673476           0.000124   
10          True          0.639139       0.666174           0.000107   
            False         0.583788       0.660073           0.000146   
11          True          0.666098       0.668040           0.000276   
            False         0.627074       0.669270           0.000248   
12          True          0.647669       0.669951           0.000312   
            False         0.590394       0.661448           0.000194   
13          True          0.663183       0.664543           0.000215   
            False         0.624881       0.668433           0.000119   
14          True          0.643207       0.663752           0.000205   
            False         0.581543       0.654505           0.000066   
15          True          0.656572       0.656547           0.000238   
            False         0.611346       0.659720           0.000118   

                          accuracy_score  explained_variance_score  
n_neighbors oversampling                                            
1           True                0.734483                 -0.064910  
            False               0.737793                 -0.050256  
2           True                0.696138                 -0.124147  
            False               0.682759                 -0.091675  
3           True                0.703034                 -0.185626  
            False               0.708414                 -0.165475  
4           True                0.694759                 -0.193804  
            False               0.688000                 -0.135826  
5           True                0.683724                 -0.257774  
            False               0.697517                 -0.206323  
6           True                0.682621                 -0.262216  
            False               0.683862                 -0.177275  
7           True                0.674483                 -0.293414  
            False               0.687034                 -0.244507  
8           True                0.678345                 -0.285454  
            False               0.674483                 -0.218142  
9           True                0.668000                 -0.317437  
            False               0.678207                 -0.275548  
10          True                0.669103                 -0.326073  
            False               0.669793                 -0.238273  
11          True                0.666069                 -0.323942  
            False               0.674207                 -0.289069  
12          True                0.672138                 -0.31527

In [22]:
best_knn_1 = knn_results_1.iloc[[
    knn_results_1.reset_index()[DECISION_METRIC].idxmax()]]
best_knn_2 = knn_results_2.iloc[[
    knn_results_2.reset_index()[DECISION_METRIC].idxmax()]]
best_knn_3 = knn_results_3.iloc[[
    knn_results_3.reset_index()[DECISION_METRIC].idxmax()]]

display(best_knn_1, best_knn_2, best_knn_3)

,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
n_neighbors,oversampling,,,,,
1,True,0.462009,0.745942,0.000359,0.727391,-0.800967


,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
n_neighbors,oversampling,,,,,
1,True,0.453041,0.691204,0.000259,0.697507,-0.787936


,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
n_neighbors,oversampling,,,,,
1,False,0.714848,0.735849,0.000147,0.737793,-0.050256


#### 4. Support Vector Classifier

In [23]:
svc_results = hyperparameter_tuning_cv(model='svc', data=X_fine, labels=y_fine.Label, cv_k=CV_K,
                                       params=SVC_PARAMS)

display(svc_results)

f1_score  roc_auc_score  roc_auc_score_var  \
kernel gamma oversampling                                               
linear 0.10  True          0.394461       0.573317       1.072627e-04   
             False         0.001411       0.500228       4.688747e-07   
rbf    0.10  True          0.078131       0.518027       7.128799e-06   
             False         0.025509       0.506435       3.623232e-06   
linear 0.01  True          0.394461       0.573317       1.072627e-04   
             False         0.001411       0.500228       4.688747e-07   
rbf    0.01  True          0.424296       0.604289       1.083964e-04   
             False         0.063399       0.513524       4.349763e-06   

                           accuracy_score  explained_variance_score  
kernel gamma oversampling                                            
linear 0.10  True                0.630908                 -0.853806  
             False               0.732049                 -0.001029  
rbf    0.10  True                0.739325                 -0.002535  
             False               0.735473                  0.007882  
linear 0.01  True                0.630908                 -0.853806  
             False               0.732049                 -0.001029  
rbf    0.01  True                0.679696                 -0.629603  
             False               0.736377                 -0.014064

In [24]:
# Best model parameters

best_svc = svc_results.iloc[[
    svc_results.reset_index()[DECISION_METRIC].idxmax()]]

display(best_svc)

,,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
kernel,gamma,oversampling,,,,,
rbf,0.01,True,0.424296,0.604289,0.000108,0.679696,-0.629603


In [25]:
svc_results_1 = hyperparameter_tuning_cv(
    model='svc', data=X_e_1, labels=y_e_1, cv_k=CV_K, params=SVC_PARAMS)
svc_results_2 = hyperparameter_tuning_cv(
    model='svc', data=X_e_2, labels=y_e_2, cv_k=CV_K, params=SVC_PARAMS)
svc_results_3 = hyperparameter_tuning_cv(
    model='svc', data=X_e_3, labels=y_e_3, cv_k=CV_K, params=SVC_PARAMS)

In [26]:
best_svc_1 = svc_results_1.iloc[[
    svc_results_1.reset_index()[DECISION_METRIC].idxmax()]]
best_svc_2 = svc_results_2.iloc[[
    svc_results_2.reset_index()[DECISION_METRIC].idxmax()]]
best_svc_3 = svc_results_3.iloc[[
    svc_results_3.reset_index()[DECISION_METRIC].idxmax()]]


display(best_svc_1, best_svc_2, best_svc_3)

,,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
kernel,gamma,oversampling,,,,,
rbf,0.01,True,0.448153,0.68052,0.000758,0.822853,-0.379382


,,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
kernel,gamma,oversampling,,,,,
rbf,0.01,True,0.279865,0.560583,0.000137,0.742257,-0.715562


,,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
kernel,gamma,oversampling,,,,,
rbf,0.01,True,0.676963,0.692577,0.000164,0.693103,-0.232


#### 5. Naive Bayes Classifier

In [27]:
nb_results = hyperparameter_tuning_cv(model='naive_bayes', data=X_fine, labels=y_fine.Label, cv_k=CV_K,
                                      params=NAIVE_BAYES_PARAMS)

display(nb_results)

,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
oversampling,,,,,
True,0.365212,0.555707,0.000126,0.627675,-0.882382
False,0.320867,0.549722,0.000174,0.669234,-0.673068


In [28]:
# Best model parameters

best_nb = nb_results.iloc[[nb_results.reset_index()[DECISION_METRIC].idxmax()]]

display(best_nb)

,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
oversampling,,,,,
True,0.365212,0.555707,0.000126,0.627675,-0.882382


In [29]:
nb_results_1 = hyperparameter_tuning_cv(
    model='naive_bayes', data=X_e_1, labels=y_e_1, cv_k=CV_K, params=NAIVE_BAYES_PARAMS)
nb_results_2 = hyperparameter_tuning_cv(
    model='naive_bayes', data=X_e_2, labels=y_e_2, cv_k=CV_K, params=NAIVE_BAYES_PARAMS)
nb_results_3 = hyperparameter_tuning_cv(
    model='naive_bayes', data=X_e_3, labels=y_e_3, cv_k=CV_K, params=NAIVE_BAYES_PARAMS)

In [30]:
best_nb_1 = nb_results_1.iloc[[
    nb_results_1.reset_index()[DECISION_METRIC].idxmax()]]
best_nb_2 = nb_results_2.iloc[[
    nb_results_2.reset_index()[DECISION_METRIC].idxmax()]]
best_nb_3 = nb_results_3.iloc[[
    nb_results_3.reset_index()[DECISION_METRIC].idxmax()]]

display(best_nb_1, best_nb_2, best_nb_3)

,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
oversampling,,,,,
True,0.348925,0.633754,0.000172,0.679903,-1.213636


,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
oversampling,,,,,
False,0.221026,0.525838,0.000164,0.722835,-0.844261


,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
oversampling,,,,,
True,0.457686,0.58399,0.000096,0.597517,-0.459625


#### 6. Decision Tree

In [31]:
dt_results = hyperparameter_tuning_cv(model='decision_tree', data=X_fine, labels=y_fine.Label, cv_k=CV_K,
                                      params=DECISION_TREE_PARAMS)

display(dt_results)

f1_score  roc_auc_score  roc_auc_score_var  \
max_depth oversampling                                               
3         True          0.353433       0.534243           0.000171   
          False         0.071236       0.509562           0.000053   
5         True          0.314374       0.540957           0.000192   
          False         0.169786       0.527595           0.000080   
7         True          0.347623       0.554038           0.000207   
          False         0.259171       0.549718           0.000144   

                        accuracy_score  explained_variance_score  
max_depth oversampling                                            
3         True                0.573657                 -1.036293  
          False               0.727294                 -0.091920  
5         True                0.650832                 -0.753702  
          False               0.722111                 -0.208965  
7         True                0.644555                 -0.794180  
          False               0.718878                 -0.305948

In [32]:
# Best model parameters

best_dt = dt_results.iloc[[dt_results.reset_index()[DECISION_METRIC].idxmax()]]

display(best_dt)

,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
max_depth,oversampling,,,,,
7,True,0.347623,0.554038,0.000207,0.644555,-0.79418


In [33]:
dt_results_1 = hyperparameter_tuning_cv(
    model='decision_tree', data=X_e_1, labels=y_e_1, cv_k=CV_K, params=DECISION_TREE_PARAMS)
dt_results_2 = hyperparameter_tuning_cv(
    model='decision_tree', data=X_e_2, labels=y_e_2, cv_k=CV_K, params=DECISION_TREE_PARAMS)
dt_results_3 = hyperparameter_tuning_cv(
    model='decision_tree', data=X_e_3, labels=y_e_3, cv_k=CV_K, params=DECISION_TREE_PARAMS)

In [34]:
best_dt_1 = dt_results_1.iloc[[
    dt_results_1.reset_index()[DECISION_METRIC].idxmax()]]
best_dt_2 = dt_results_2.iloc[[
    dt_results_2.reset_index()[DECISION_METRIC].idxmax()]]
best_dt_3 = dt_results_3.iloc[[
    dt_results_3.reset_index()[DECISION_METRIC].idxmax()]]

display(best_dt_1, best_dt_2, best_dt_3)

,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
max_depth,oversampling,,,,,
5,True,0.364311,0.643099,0.000583,0.7141,-1.05951


,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
max_depth,oversampling,,,,,
7,True,0.273827,0.546089,0.000566,0.678871,-1.073322


,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
max_depth,oversampling,,,,,
7,True,0.60851,0.62356,0.000353,0.623586,-0.500418


#### 7. Random Forest

In [35]:
rf_results = hyperparameter_tuning_cv(model='random_forest', data=X_fine, labels=y_fine.Label, cv_k=CV_K,
                                      params=RANDOM_FOREST_PARAMS)

display(rf_results)

f1_score  roc_auc_score  \
max_depth n_estimators oversampling                            
3         3            True          0.380386       0.550443   
                       False         0.046411       0.508465   
          5            True          0.381599       0.552931   
                       False         0.017054       0.502853   
          7            True          0.384961       0.555736   
                       False         0.022973       0.504501   
5         3            True          0.364082       0.549755   
                       False         0.088593       0.514143   
          5            True          0.381206       0.566696   
                       False         0.067448       0.512423   
          7            True          0.369134       0.565273   
                       False         0.054685       0.509899   
7         3            True          0.357228       0.552378   
                       False         0.147702       0.522405   
          5            True          0.365171       0.560417   
                       False         0.128833       0.523422   
          7            True          0.377090       0.573426   
                       False         0.098576       0.517924   

                                     roc_auc_score_var  accuracy_score  \
max_depth n_estimators oversampling                                      
3         3            True                   0.000155        0.582073   
                       False                  0.000013        0.733096   
          5            True                   0.000234        0.589444   
                       False                  0.000013        0.732192   
          7            True                   0.000102        0.592534   
                       False                  0.000004        0.733191   
5         3            True                   0.000069        0.612173   
                       False                  0.000012        0.729957   
          5            True                   0.000115        0.633619   
                       False                  0.000009        0.733381   
          7            True                   0.000155        0.645792   
                       False                  0.000009        0.733096   
7         3            True                   0.000181        0.629862   
                       False                  0.000028        0.723205   
          5            True                   0.000113        0.641512   
                       False                  0.000058        0.731764   
          7            True                   0.000141        0.661246   
                       False                  0.000030        0.732810   

                                     explained_variance_score  
max_depth n_estimators oversampling                            
3         3            True                         -1.012063  
                       False                        -0.026826  
          5            True                         -1.004161  
                       False                        -0.011845  
          7            True                         -0.991476  
                       False                        -0.008006  
5         3            True                         -0.940715  
                       False                        -0.083985  
          5            True                         -0.844661  
                       False                        -0.040529  
          7            True                         -0.790771  
                       False                        -0.033073  
7         3            True                         -0.876227  
                       False                        -0.184197  
          5            True                         -0.819651  
                       False                        -0.102183  
          7            True                         -0.723616  
                       False                        -0.0696

In [36]:
# Best model parameters

best_rf = rf_results.iloc[[rf_results.reset_index()[DECISION_METRIC].idxmax()]]

display(best_rf)

,,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
max_depth,n_estimators,oversampling,,,,,
7,7,True,0.37709,0.573426,0.000141,0.661246,-0.723616


In [37]:
rf_results_1 = hyperparameter_tuning_cv(
    model='random_forest', data=X_e_1, labels=y_e_1, cv_k=CV_K, params=RANDOM_FOREST_PARAMS)
rf_results_2 = hyperparameter_tuning_cv(
    model='random_forest', data=X_e_2, labels=y_e_2, cv_k=CV_K, params=RANDOM_FOREST_PARAMS)
rf_results_3 = hyperparameter_tuning_cv(
    model='random_forest', data=X_e_3, labels=y_e_3, cv_k=CV_K, params=RANDOM_FOREST_PARAMS)

In [38]:
best_rf_1 = rf_results_1.iloc[[
    rf_results_1.reset_index()[DECISION_METRIC].idxmax()]]
best_rf_2 = rf_results_2.iloc[[
    rf_results_2.reset_index()[DECISION_METRIC].idxmax()]]
best_rf_3 = rf_results_3.iloc[[
    rf_results_3.reset_index()[DECISION_METRIC].idxmax()]]

display(best_rf_1, best_rf_2, best_rf_3)

,,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
max_depth,n_estimators,oversampling,,,,,
7,5,True,0.38839,0.651795,0.000474,0.771799,-0.737286


,,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
max_depth,n_estimators,oversampling,,,,,
5,5,True,0.297729,0.557786,0.000509,0.66916,-1.130569


,,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
max_depth,n_estimators,oversampling,,,,,
7,7,False,0.584217,0.631952,0.000181,0.637103,-0.437801


#### 8. Gradient Boosting

In [39]:
gb_results = hyperparameter_tuning_cv(model='gradient_boosting', data=X_fine, labels=y_fine.Label, cv_k=CV_K,
                                      params=GRADIENT_BOOSTING_PARAMS)

display(gb_results)

f1_score  roc_auc_score  \
max_depth n_estimators oversampling                            
3         3            True          0.357928       0.553579   
                       False         0.000000       0.500000   
          5            True          0.358746       0.558244   
                       False         0.001037       0.500260   
          7            True          0.358379       0.558080   
                       False         0.001791       0.500449   
5         3            True          0.334291       0.559778   
                       False         0.000000       0.500000   
          5            True          0.314696       0.557451   
                       False         0.005380       0.501165   
          7            True          0.316430       0.561197   
                       False         0.018583       0.503523   
7         3            True          0.330017       0.558366   
                       False         0.001417       0.500258   
          5            True          0.328702       0.564919   
                       False         0.033933       0.506921   
          7            True          0.328637       0.568281   
                       False         0.076665       0.516482   

                                     roc_auc_score_var  accuracy_score  \
max_depth n_estimators oversampling                                      
3         3            True               1.849468e-04        0.629434   
                       False              0.000000e+00        0.732049   
          5            True               2.289138e-04        0.642939   
                       False              3.014723e-07        0.732192   
          7            True               1.360000e-04        0.643034   
                       False              2.019453e-07        0.732287   
5         3            True               1.926142e-04        0.678697   
                       False              0.000000e+00        0.732049   
          5            True               2.244279e-04        0.694341   
                       False              1.484364e-06        0.732477   
          7            True               2.820746e-04        0.701617   
                       False              8.433978e-06        0.732763   
7         3            True               9.419283e-05        0.679410   
                       False              2.238332e-07        0.732097   
          5            True               1.368759e-04        0.698573   
                       False              7.189776e-06        0.733999   
          7            True               1.827690e-04        0.707656   
                       False              2.336286e-05        0.737090   

                                     explained_variance_score  
max_depth n_estimators oversampling                            
3         3            True                         -0.859394  
                       False                         0.000000  
          5            True                         -0.812802  
                       False                         0.000324  
          7            True                         -0.810418  
                       False                         0.000574  
5         3            True                         -0.613167  
                       False                         0.000000  
          5            True                         -0.515751  
                       False                        -0.000497  
          7            True                         -0.470773  
                       False                        -0.007993  
7         3            True                         -0.611918  
                       False                        -0.000665  
          5            True                         -0.496998  
                       False                        -0.009804  
          7            True                         -0.438996  
                       False                        -0.0183

In [40]:
# Best model parameters

best_gb = gb_results.iloc[[gb_results.reset_index()[DECISION_METRIC].idxmax()]]

display(best_gb)

,,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
max_depth,n_estimators,oversampling,,,,,
7,7,True,0.328637,0.568281,0.000183,0.707656,-0.438996


In [41]:
gb_results_1 = hyperparameter_tuning_cv(
    model='gradient_boosting', data=X_e_1, labels=y_e_1, cv_k=CV_K, params=GRADIENT_BOOSTING_PARAMS)
gb_results_2 = hyperparameter_tuning_cv(
    model='gradient_boosting', data=X_e_2, labels=y_e_2, cv_k=CV_K, params=GRADIENT_BOOSTING_PARAMS)
gb_results_3 = hyperparameter_tuning_cv(
    model='gradient_boosting', data=X_e_3, labels=y_e_3, cv_k=CV_K, params=GRADIENT_BOOSTING_PARAMS)

In [42]:
best_gb_1 = gb_results_1.iloc[[
    gb_results_1.reset_index()[DECISION_METRIC].idxmax()]]
best_gb_2 = gb_results_2.iloc[[
    gb_results_2.reset_index()[DECISION_METRIC].idxmax()]]
best_gb_3 = gb_results_3.iloc[[
    gb_results_3.reset_index()[DECISION_METRIC].idxmax()]]
display(best_gb_1, best_gb_2, best_gb_3)

,,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
max_depth,n_estimators,oversampling,,,,,
5,5,True,0.404561,0.667325,0.00041,0.768071,-0.746631


,,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
max_depth,n_estimators,oversampling,,,,,
7,5,True,0.290295,0.567149,0.000624,0.750525,-0.65415


,,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
max_depth,n_estimators,oversampling,,,,,
7,7,False,0.635139,0.677786,0.000187,0.682621,-0.250957


### Results

In [43]:
# display('logistic', best_log)
# display('lda', best_lda)
# display('knn', best_knn)
# display('svc', best_svc)
# display('naive_bayes', best_nb)
# display('decision_tree', best_dt)
# display('random_forest', best_rf)
# display('gradient_boosting', best_gb)

In [44]:
weighted_average = (np.array([len(X_e_1), len(X_e_2), len(X_e_3)]) / len(X_e))

e_log = np.sum(pd.concat([best_log_1, best_log_2, best_log_3]
                         ).roc_auc_score.values * weighted_average)
e_lda = np.sum(pd.concat([best_lda_1, best_lda_2, best_lda_3]
                         ).roc_auc_score.values * weighted_average)
e_knn = np.sum(pd.concat([best_knn_1, best_knn_2, best_knn_3]
                         ).roc_auc_score.values * weighted_average)
e_svc = np.sum(pd.concat([best_svc_1, best_svc_2, best_svc_3]
                         ).roc_auc_score.values * weighted_average)
e_nb = np.sum(pd.concat([best_nb_1, best_nb_2, best_nb_3]
                        ).roc_auc_score.values * weighted_average)
e_dt = np.sum(pd.concat([best_dt_1, best_dt_2, best_dt_3]
                        ).roc_auc_score.values * weighted_average)
e_rf = np.sum(pd.concat([best_rf_1, best_rf_2, best_rf_3]
                        ).roc_auc_score.values * weighted_average)
e_gb = np.sum(pd.concat([best_gb_1, best_gb_2, best_gb_3]
                        ).roc_auc_score.values * weighted_average)

In [45]:
# Results expert features
# First row is Expert 1, second Expert 2, third Expert 3
# display('logistic', pd.concat([best_log_1,best_log_2,best_log_3]))
# display('lda', pd.concat([best_lda_1,best_lda_2,best_lda_3]))
display('knn', pd.concat([best_knn_1, best_knn_2, best_knn_3]))
# display('svc', pd.concat([best_svc_1,best_svc_2,best_svc_3]))
# display('naive_bayes', pd.concat([best_nb_1,best_nb_2,best_nb_3]))
# display('decision_tree', pd.concat([best_dt_1,best_dt_2,best_dt_3]))
# display('random_forest', pd.concat([best_rf_1,best_rf_2,best_rf_3]))
# display('gradient_boosting', pd.concat([best_gb_1,best_gb_2,best_gb_3]))

'knn'

f1_score  roc_auc_score  roc_auc_score_var  \
n_neighbors oversampling                                               
1           True          0.462009       0.745942           0.000359   
            True          0.453041       0.691204           0.000259   
            False         0.714848       0.735849           0.000147   

                          accuracy_score  explained_variance_score  
n_neighbors oversampling                                            
1           True                0.727391                 -0.800967  
            True                0.697507                 -0.787936  
            False               0.737793                 -0.050256

In [46]:
results = pd.DataFrame(data={'models': ['logistic', 'lda', 'knn', 'svc', 'naive_bayes', 'decision_tree',
                                        'random_forest', 'gradient_boosting'],
                             'auc_best': [best_log.roc_auc_score.values[0], best_lda.roc_auc_score.values[0],
                                          best_knn.roc_auc_score.values[0], best_svc.roc_auc_score.values[0],
                                          best_nb.roc_auc_score.values[0], best_dt.roc_auc_score.values[0],
                                          best_rf.roc_auc_score.values[0], best_gb.roc_auc_score.values[0]],
                             'auc_expert_weighted': [e_log, e_lda, e_knn, e_svc, e_nb, e_dt, e_rf, e_gb]})

display(results)

,models,auc_best,auc_expert_weighted
0,logistic,0.577510,0.613839
1,lda,0.577981,0.613497
2,knn,0.679682,0.722642
3,svc,0.604289,0.641246
4,naive_bayes,0.555707,0.577524
5,decision_tree,0.554038,0.601236
6,random_forest,0.573426,0.610914
7,gradient_boosting,0.568281,0.634657


### Conclusions


In [47]:
# Save dataframe
results.to_pickle("results_fine_metadata.pkl")

In [48]:
results.mean(axis=0)

auc_best               0.586364
auc_expert_weighted    0.626944
dtype: float64

In [49]:
results.loc[results['auc_best'].argmax()]

models                      knn
auc_best               0.679682
auc_expert_weighted    0.722642
Name: 2, dtype: object

In [50]:
results.loc[results['auc_expert_weighted'].argmax()]

models                      knn
auc_best               0.679682
auc_expert_weighted    0.722642
Name: 2, dtype: object